# Lab 07.2: Distance Metrics (15 Points Possible Bonus!)

This lab is presented with some revisions from [Dennis Sun at Cal Poly](https://web.calpoly.edu/~dsun09/index.html) and his [Data301 Course](http://users.csc.calpoly.edu/~dsun09/data301/lectures.html)

### When you have filled out all the questions, submit via [Tulane Canvas](https://tulane.instructure.com/)

The previous labs we discussed ways to measure relationships between variables, or the _columns_ of a `DataFrame`. This chapter is about how to measure relationships between observations, or the _rows_ of a `DataFrame`. How do we quantify how "similar" two observations are? We will use the Ames housing data set, but to keep things simple, we will work with just three quantitative variables from that data set: the number of bedrooms, the number of bathrooms, and the living area (in square feet).

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

housing_df = pd.read_csv("../data/ames.tsv", sep="\t")

# extract 3 quantitative variables
housing_df_quant = housing_df[["Bedroom AbvGr", "Gr Liv Area"]].copy()
housing_df_quant["Bathrooms"] = (
    housing_df["Full Bath"] + 
    0.5 * housing_df["Half Bath"]
)
housing_df_quant

Shown below is a (three-dimensional) scatterplot of these variables. Consider the two observations connected by a red line. (The label next to each point is its index in the `DataFrame`.) To measure how similar they are, we can calculate the distance between the two points.

<img src="../images/distance.png">

Calculating the distance between two points is not as straightforward as it might seem because there is more than one way to define distance. The one most familiar to you is probably **Euclidan distance**, which is the straight-line distance ("as the crow flies") between the two points. The formula for calculating this distance is a generalization of the Pythagorean theorem:

$$ d({\bf x}, {\bf x'}) = \sqrt{\sum_{j=1}^D (x_j - x'_j)^2} $$

Which we've seen before as the sum of squared distances!

In [ ]:
x = housing_df_quant.loc[2927]
x1 = housing_df_quant.loc[2928]

x - x1

In [ ]:
(x - x1) ** 2

In [ ]:
np.sqrt(((x - x1) ** 2).sum())

The beauty of this definition is that it generalizes to more than three dimensions. Even though it is difficult to visualize points in 100-dimensional space, we can calculate distances between them in exactly the same way.

However, Euclidean distance is not the only way to measure how far apart two points are. There is also [**Manhattan distance**](https://en.wikipedia.org/wiki/Taxicab_geometry) (also called _taxicab distance_ ), which measures the distance a taxicab in Manhattan would have to drive to travel from A to B. Taxicabs are not able to travel in a straight line (i.e., the green path below, the Euclidian distance) because they have to follow the street grid. But there are multiple paths along the street grid that all have exactly the same length (i.e., the red, yellow, and blue paths below); the Manhattan distance is the length of any one of these shortest paths.

<img src="../images/dist.png">

The formula for Manhattan distance is actually quite similar to the formula for Euclidean distance. Instead of squaring the differences and taking the square root at the end (as in Euclidean distance), we simply take absolute values:
$$ d({\bf x}, {\bf x'}) = \sum_{j=1}^D |x_j - x'_j|. $$

The following code calculates Manhattan distance:

In [ ]:
((x - x1).abs()).sum()

### Comparison of Euclidean and Manhattan distance

The Euclidean distance was essentially just the largest difference. This is because Euclidean distance first _squares_ the differences. The squaring operation has a "rich get richer" effect; larger values get magnified by more than smaller values. As a result, the largest differences tend to dominate the Euclidean distance.

On the other hand, Manhattan distance treats all differences equally. So Manhattan distance is preferred if you are concerned that an outlier in one variable might dominate the distance metric.

## The Importance of Scaling

Here's a quiz. There are two pairs of observations in the figure below, one connected by a red line, the other connected by an orange line. Which pair of observations is more similar (assuming we use Euclidean distance)?

![](../images/closer.png)

Let's actually calculate these two distances.

In [ ]:
# Distance between two points connected by red line
x = housing_df_quant.loc[2927]
x1 = housing_df_quant.loc[2928]

np.sqrt(((x - x1) ** 2).sum())

In [ ]:
# Distance between two points connected by orange line
x = housing_df_quant.loc[2498]
x1 = housing_df_quant.loc[290]

np.sqrt(((x - x1) ** 2).sum())

Surprised by the answer? The scatterplot is deceiving because it automatically scales the variables to make the points fit on the same plot. In reality, the variables are on very different scales. The number of bedrooms and bathrooms range from 0 to 6, while living area is in the thousands. When variables are on such different scales, the variable with the largest variability will dominate the distance metric.

The plot below shows the same data, but drawn to scale. You can see that differences in the number of bedrooms and the number of bathrooms hardly matter at all; only the variability in the living area matters.

![](../images/closer_rescaled.png)

To obtain distances that agree more with our intuition---and that do not give too much weight to any one variable---we transform the variables to be on the same scale. There are a few ways to **scale** a variable:

- **standardizing**: subtract each variable by its mean, then divide by its standard deviation, (also called z-standardization, 
$$ x_i \leftarrow \frac{x_i - \text{mean}[X]}{\text{SD}[X]} $$
- **normalizing**: scale each variable to have length (or "norm") 1, 
$$ x_i \leftarrow \frac{x_i}{\sqrt{\sum_{i=1}^n x_i^2}} $$
- **min/max scaling**: scale each variable so that all values are between 0 and 1, 
$$x_i \leftarrow \frac{x_i - \min[X]}{\max[X] - \min[X]}.$$

The figure below illustrates what each of these scaling methods do to a synthetic data set with two variables. All three methods scale the variables in similar (but slightly different) ways, resulting in figure-eights with different aspect ratios.  Standardizing also moves the data to be centered around the origin, while min-max scaling moves the data to be in a box whose corners are $(0, 0)$ and $(1, 1)$.

![](../images/scaling.png)

Let's standardize the Ames housing data, and see how it affects the distance metric.

In [ ]:
housing_df_std = (
    (housing_df_quant - housing_df_quant.mean()) / 
    housing_df_quant.std()
)
housing_df_std

Notice that the resulting `DataFrame` contains negative values. This makes sense because standardizing makes the mean of every variable equal to 0. If the mean is 0, then some values must be negative.

The above command is deceptively simple. We actually subtracted a `DataFrame` by a `Series`, then divided the resulting `DataFrame` by another `Series`. We relied on `pandas` to broadcast each `Series` over the right dimension of the `DataFrame`. To be more explicit about the broadcasting, we could have also used the `.sub()` and `.divide()` methods (instead of `-` and `/`) and been explicit about the axis:

In [ ]:
housing_df_std = (housing_df_quant.
                  sub(housing_df_quant.mean(), axis=1).
                  divide(housing_df_quant.std(), axis=1))
housing_df_std

Now let's recalculate the distances using this standardized data and see if our conclusions change.

In [ ]:
# Distance between two points connected by red line
x = housing_df_std.loc[2927]
x1 = housing_df_std.loc[2928]

np.sqrt(((x - x1) ** 2).sum())

In [ ]:
# Distance between two points connected by orange line
x = housing_df_std.loc[2498]
x1 = housing_df_std.loc[290]

np.sqrt(((x - x1) ** 2).sum())

So, if we first standardize the data, then the pair of observations connected by the red line are more similar than the pair connected by the orange line, which matches our intuition. It is (almost) always a good idea to scale your variables before calculating distances.

Now that you've seen how to implement one scaling method (standardization), you will implement two more (normalization and min-max scaling) in Exercises 1 and 2 below.

# Part 1 Exercises

**Exercise 1.** Instead of standardizing the three variables from the Ames housing data set, normalize them. Then, recompute the distances between the two pairs of points above. Does your conclusion change?

In [ ]:
# YOUR CODE HERE

**Written Answers Here:**

**Exercise 2.** Instead of standardizing the three variables from the Ames housing data set, apply a min-max scaling to them. Then, recompute the distances between the two pairs of points above. Does your conclusion change?

In [ ]:
# YOUR CODE HERE

**Written Answers Here:**

Exercises 3-5 ask you to work with a data set that describes the chemical composition of 1599 red wines (`../data/reds.csv`). There are 12 variables in this data set, all of which are quantitative (so each observation is a point in 12-dimensional space).

In [ ]:
df_reds = pd.read_csv("../data/reds.csv", sep=';')
df_reds[:5]

**Exercise 3.** Find which red wine is more similar to wine 0 in the `DataFrame`: wine 6 or wine 36? (Do not scale the variables.) You should do this for both Euclidian Distance and Manhatten Distance.  Does your answer depend on which distance metric you use to measure "similarity"?

In [ ]:
# YOUR CODE HERE

**Written Answers:**

**Exercise 4.** Now suppose we agree to measure similarity using Euclidean distance, and we wish to investigate the effect of scaling the variables. Which red wine is more similar to wine 0: wine 6 or wine 36? Does the answer depend on whether the variables are scaled or not? Does it depend on the choice of scaling?  What happens for each type of scaling?

In [ ]:
# YOUR CODE HERE

**Written Answers Here:**

# Part 2: Distances Between Categorical Variables

The distance metrics that we studied in the previous section were designed for quantitative variables. But most data sets contain a mix of categorical and quantitative variables. For example, the Titanic data set contains both quantitative variables, like `age`, and categorical variables, like `sex` and `embarked`. How do we measure the similarity between observations for a data set like this one? The most straightforward solution is to convert the categorical variables into quantitative ones.

In [ ]:
titanic = pd.read_csv("../data/titanic.csv")
titanic

## Converting Categorical Variables to Quantitative Variables

Binary categorical variables (categorical variables with two categories) can be converted into quantitative variables by coding one category as 1 and the other category as 0. (In fact, the `survived` column in the Titanic data set is an example of a variable where this has been done.) But what do we do about a categorical variable with more than 2 categories, like `embarked`, which has 3 categories?

We can convert a categorical variable with $K$ categories into $K$ separate 0/1 variables, or **dummy variables**. Each of the $K$ variables is an indicator for one of the $K$ categories. That is, each dummy variable is 1 if the observation fell into that category and 0 otherwise.

Although it is not difficult to create dummy variables manually, the easiest way to create them is the `get_dummies()` function in `pandas`.

In [ ]:
pd.get_dummies(titanic["embarked"])

Since every observation is in exactly one category, each row contains exactly one 1; the rest of the values in each row are 0s.

We can call `get_dummies` on a `DataFrame` to encode multiple categorical variables at once. `pandas` will only dummy-encode the variables it deems categorical, leaving the quantitative variables alone. If there are any categorical variables that are represented in the `DataFrame` using numeric types, they must be cast explicitly to a categorical type, such as `str`.  `pandas` will also automatically prepend the variable name to all dummy variables, to prevent collisions between column names in the final `DataFrame`.

In [ ]:
# Convert pclass to a categorical type
titanic["pclass"] = titanic["pclass"].astype(str)

# Pass all variables to get_dummies, except ones that are "other" types
titanic_num = pd.get_dummies(
    titanic.drop(["name", "ticket", "cabin", "boat", "body"], axis=1)
)
titanic_num

Notice that categorical variables, like `pclass`, were converted to dummy variables with names like `pclass_1`, `pclass_2` and `pclass_3`, while quantitative variables, like `age`, were left alone.

Now that we have converted every variable in our data set into a quantitative variable, we can apply the techniques from the previous section to calculate distances between observations. For example, to find the passenger who is most similar to the first passenger, Elisabeth Watson, we can find the row with the smallest Euclidean distance to that row in the above `DataFrame`.

In [ ]:
titanic_std = (titanic_num - titanic_num.mean()) / titanic_num.std()
np.sqrt(
    ((titanic_std - titanic_std.loc[0]) ** 2).sum(axis=1)
).sort_values()

The passenger who was most similar to Elisabeth Allen, other than herself, is passenger 238. Let's extract these passengers from the original `DataFrame` to see how similar they really are.

In [ ]:
titanic.loc[[0, 238]]

The two passengers are indeed very similar, only differing in age and the number of parents/children accompanying her. They even happen to share the same first two names ("Elizabeth Walton").

# Exercises Pt. 2 (Bonus 7 Points)

Exercises 1 and 2 use the Ames housing data set (`../data/ames.tsv`).

In [ ]:
df_ames = pd.read_csv("../data/ames.tsv", sep="\t")
df_ames[:5]

**Bonus Exercise 1 (2 Points).** The neighborhood variable (`Neighborhood`) in this data set is categorical. Convert it to $K$ quantitative variables. What is $K$ in this case?

Based on these $K$ variables only, calculate the Euclidean distance between house 0 and each of the other houses in the data set. What are the possible values of the Euclidean distance? Can you explain what a distance of $0$ means, in the context of this variable? What about a distance of $1$?

In [ ]:
# ENTER YOUR CODE HERE

**Written Answers**:

**Bonus Exercise 2 (5 Points).** Suppose that you really like house 0 in the data set, but it is too expensive. Find cheaper homes that are similar to it, by calculating distances after encoding categorical variables as dummy variables. Be sure to actually look at the profiles of the homes that your algorithm picked out as most similar. Do they make sense?

Try different distance metrics and different standardization methods. How sensitive are your results to these choices?

_Think:_ If the goal is to find a "good deal" on a similar house, should sale price be included as a variable in your distance metric? 

_Hint:_ There are too many variables in the data set. Do not try to call `pd.get_dummies()` on the entire `DataFrame`! You will want to pare down the number of variables, but be sure to include a mixture of categorical and quantitative variables. Refer to the [data documentation](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) for information about the variables.

In [ ]:
# ENTER YOUR CODE HERE

**Written Answers:**

# Part 3: The Distance Matrix

In many applications, we need the distance between every pair of observations ${\bf x}_i$ and ${\bf x}_j$ in a data set. How do we represent this information? The most common way is to use an $n \times n$ matrix, where the $(i, j)$th entry is the distance between ${\bf x}_i$ and ${\bf x}_j$. That is,

$$ D = \begin{pmatrix} 
d({\bf x}_1, {\bf x}_1) & d({\bf x}_1, {\bf x}_2) & \cdots & d({\bf x}_1, {\bf x}_n) \\ 
d({\bf x}_2, {\bf x}_1) & d({\bf x}_2, {\bf x}_2) & \cdots & d({\bf x}_2, {\bf x}_n) \\ 
\vdots & \vdots & \ddots & \vdots \\
d({\bf x}_n, {\bf x}_1) & d({\bf x}_n, {\bf x}_2) & \cdots & d({\bf x}_n, {\bf x}_n)
\end{pmatrix}. $$

There are a few things we can say about the $n\times n$ distance matrix $D$.

1. All of the entries of $D$ are non-negative.
2. Because the distance between any observation and itself, $d({\bf x}_i, {\bf x}_i)$, is always zero, the _diagonal_ elements of this matrix, $D_{ii}$ are all equal to 0.
3. For many distance metrics, including Euclidean and Manhattan distance, $d$ is symmetric, meaning that $d({\bf x}_i, {\bf x}_j) = d({\bf x}_i, {\bf x}_j)$. Therefore, the matrix $D$ will also be symmetric; that is, the values in the upper triangle will match their reflection in the lower triangle.

How do we calculate the distance matrix for a `DataFrame` consisting of all quantitative variables? For example, suppose we want to calculate the matrix of distances between each of the houses in the Ames housing data set, based on the number of bedrooms, number of bathrooms, and the living area (in square feet).

In [ ]:
housing_df = pd.read_csv("../data/ames.tsv",sep="\t")

# extract 3 quantitative variables
housing_df_quant = housing_df[["Bedroom AbvGr", "Gr Liv Area"]].copy()
housing_df_quant["Bathrooms"] = (
    housing_df["Full Bath"] + 
    0.5 * housing_df["Half Bath"]
)
housing_df_quant

_The Long Way:_ It is possible to create the distance matrix entirely in `pandas`. The idea is to first define a function that calculates the distances between a given observation and all of the other observations:

In [ ]:
def get_euclidean_dists_from_obs(obs):
    return np.sqrt(
        ((housing_df_quant - obs) ** 2).sum(axis=1)
    )

get_euclidean_dists_from_obs(housing_df_quant.loc[0])

The code for this function is very similar to the code that we wrote for Exercise 5 at the end of Part 1.

Now, to get a matrix of distances $D$, we simply need to apply this function to every row of the `DataFrame`. To achieve this, we use the `.apply()` method with `axis=1`:

In [ ]:
D = housing_df_quant.apply(
    get_euclidean_dists_from_obs,
    axis=1
)
D

Notice that this is a $2930 \times 2930$ symmetric matrix of non-negative numbers, with zeroes along the diagonal, just as we predicted.

## Better, the short way...

_The Short Way_ : There are many packages in Python that calculate distance matrices. One such package is scikit-learn, a machine learning package in Python. Machine learning will be discussed in depth in the coming Labs, and we will explore the features of scikit-learn extensively in those chapters. Because distance matrices are important in machine learning, scikit-learn provides functions for calculating distance matrices.

For example, the following code calculates the (Euclidean) distance matrix between all of the houses in the Ames housing data set:

In [ ]:
from sklearn.metrics import pairwise_distances

D_ = pairwise_distances(housing_df_quant, metric="euclidean")
D_

Notice that the return type is a `numpy` array, instead of a `pandas` `DataFrame`. That is because scikit-learn was designed to work with `numpy` arrays. Although it will accept `pandas` `DataFrame`s as arguments, scikit-learn will convert them `numpy` arrays underneath the hood and return `numpy` arrays.

Fortunately, many of the usual `pandas` operations work on `numpy` arrays as well. For example, to get the maximum value in each row, we can use the `.max()` method with `axis=1`.

In [ ]:
D_.max(axis=1)

# Exercises Pt. 3 (Bonus 8 Points)

Exercises 1-3 ask you to work with a data set that describes the chemical composition of 1599 red wines (`../data/reds.csv`). All 12 variables in this data set are quantitative.

In [ ]:
df_reds = pd.read_csv("../data/reds.csv", sep=";")
df_reds[:5]

**Bonus Exercise 3 (2 Points).** Calculate the distance between every pair of wines in this data set.

In [ ]:
# YOUR CODE HERE

**Bonus Exercise 4 (4 Points).** Using the distance matrix that you calculated in Exercise 1, calculate the distance of the wine that is most similar to each wine.

*Hint:* It might be good to think about what the [values on the diagonal](https://numpy.org/doc/stable/reference/generated/numpy.fill_diagonal.html) of the matrix are... you don't want to select the wine that is itself... you want another wine...

In [ ]:
# YOUR CODE HERE

**Bonus Exercise 5 (2 Points).** Using the distance matrix that you calculated in Exercise 1, determine the identity of the wine that is most similar to each wine.

In [ ]:
# YOUR CODE HERE

### When you have filled out all the questions, submit via [Tulane Canvas](https://tulane.instructure.com/)